# RAG JDR - Pipeline d'Ingestion

Ce notebook implémente la phase d'ingestion du système RAG (Retrieval Augmented Generation). Il transforme des résumés de sessions narratifs en une base de connaissances structurée et vectorisée.

### **Principe**
L'objectif est de convertir le texte brut des chroniques en deux collections distinctes :

* **Events :** Une mémoire chronologique des actions et rebondissements.

* **Entities :** Un registre encyclopédique des personnages, lieux et objets dont les fiches évoluent au fil des sessions.

### **Pipeline d'ingestion**

```
Texte Brut (Session X)
                |
                v
        [LLM Extraction]
                |
                v
      { Est-ce un JSON valide ? }
         /            \
      [OUI]          [NON] --> [LLM Repair] --+
        |                                     |
        v <-----------------------------------+
        |
        v
[Prévisualisation (Pandas)] <--- HUMAN-IN-THE-LOOP
(Vérification visuelle / Correction manuelle du dict)
        |
        v
  [Appel de save_session()]
        |
        v
[Distribution des données]
        |
        +---> ÉVÉNEMENTS : Indexation chronologique (ex: s001_e012)
        |
        +---> ENTITÉS : Fusion sémantique et mise à jour de l'historique
                |
                v
           [ChromaDB] <--- Stockage persistant des Embeddings
```

### **Points clés**
* **Extraction Intelligente :** Le LLM identifie les relations complexes (qui participe à quel événement).

* **Historique Cumulative :** Les fiches d'entités conservent la trace de leur évolution (ex: [S1] Allié... [S3] Traître...).

* **Cohérence Nominale :** Réinjection des entités connues dans le prompt pour éviter les doublons.

## 1. Installation des dépendances

* `openai` : Client pour l'API Gemini (compatible avec le standard OpenAI).

* `chromadb` : Moteur de recherche vectoriel pour le stockage persistant.

* `sentence-transformers` : Génère les vecteurs numériques à partir du texte.

In [93]:
!pip install -q openai chromadb sentence-transformers

## 2. Montage Google Drive

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. Configuration du provider LLM

Paramètre|Valeur
---------|-------
Modèle|gemini-2.5-flash-preview-09-2025
Température|0 (pour une extraction déterministe)
Format|JSON Mode

In [95]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
MODEL = "gemini-2.5-flash-preview-09-2025"

print(f"Provider : Gemini 2.5 Flash Preview")
print(f"Modele   : {MODEL}")

Provider : Gemini 2.5 Flash Preview
Modele   : gemini-2.5-flash-preview-09-2025


## 4. Fonction de génération (API)

In [96]:
from openai import OpenAI
import time

# Configuration du client pour l'API Gemini compatible OpenAI
client = OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=GEMINI_API_KEY,
)


def generate(prompt: str, system_prompt: str = "", max_new_tokens: int = 20000) -> str:
    """Appelle le LLM via API compatible OpenAI."""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={ "type": "json_object" },
        max_tokens=max_new_tokens,
        temperature=0,
    )

    return response.choices[0].message.content


# Test rapide
start = time.time()
result = generate("Résume en 3 phrases ce qu'est un jeu de rôle.")
elapsed = time.time() - start

print(f"Temps : {elapsed:.1f}s")
print(f"Réponse : {result[:200]}...")

Temps : 3.3s
Réponse : {
  "resume": "Un jeu de rôle (JdR) est une forme de narration collaborative où les participants incarnent des personnages fictifs au sein d'un univers imaginaire. Un joueur, appelé Maître de Jeu (MJ)...


## 5. Configuration ChromaDB

In [97]:
import chromadb
from chromadb.utils import embedding_functions

DB_PATH = "/content/drive/MyDrive/rag_jdr_db"

chroma_client = chromadb.PersistentClient(path=DB_PATH)

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large",
    device="cuda" if __import__('torch').cuda.is_available() else "cpu",
)

events_collection = chroma_client.get_or_create_collection(
    name="events",
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"},
)

entities_collection = chroma_client.get_or_create_collection(
    name="entities",
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"},
)

print(f"Events: {events_collection.count()} | Entities: {entities_collection.count()}")

Events: 53 | Entities: 72


## 6. Prompts d'extraction

In [98]:
SYSTEM_PROMPT = """Tu es un archiviste-chroniqueur expert pour les campagnes de jeu de rôle.
Ta mission est d'extraire et de rédiger des fiches d'une qualité littéraire irréprochable : phrases fluides, vocabulaire riche et précis, formulations évocatrices.
Tu réponds UNIQUEMENT en JSON valide, sans texte avant ni après, sans blocs markdown (```json)."""

EXTRACTION_TEMPLATE = '''Voici le résumé de la session {session_number} d'une campagne de jeu de rôle :
---
{resume}
---

{existing_entities_block}

## INSTRUCTIONS D'EXTRACTION

### ÉVÉNEMENTS
Extrais TOUS les événements significatifs dans leur ordre chronologique.
Chaque événement reçoit un numéro d'ordre (1, 2, 3...) reflétant sa position dans la session.
La description doit être factuelle, autonome (compréhensible sans contexte), et rédigée avec soin.

**Types d'événements (en français) :**
- `combat` : affrontement, embuscade, escarmouche
- `mort` : décès d'un personnage
- `rencontre` : première interaction avec un PNJ ou une créature
- `dialogue` : conversation importante, négociation, interrogatoire
- `révélation` : information cruciale apprise, secret découvert
- `découverte` : lieu, objet ou indice trouvé
- `quête_début` : nouvelle mission acceptée ou découverte
- `quête_fin` : mission accomplie ou échouée
- `voyage` : déplacement significatif entre deux lieux
- `repos` : halte, nuit à l'auberge, récupération
- `rêve` : vision, songe, présage
- `acquisition` : objet important obtenu (achat, don, butin)
- `perte` : objet perdu, volé ou détruit
- `autre` : événement ne rentrant dans aucune catégorie

### ENTITÉS
Extrais TOUTES les entités mentionnées ou impliquées.

**Types :**
- `pj` : personnages joueurs
- `pnj` : personnages non-joueurs
- `lieu` : villes, bâtiments, régions, donjons
- `faction` : guildes, organisations, armées, ordres
- `objet` : objets magiques ou importants pour l'intrigue
- `quête` : missions en cours ou terminées
- `créature` : monstres, animaux, êtres surnaturels
- `divinité` : dieux, saints, entités cosmiques

**Statuts :**
- `actif` : vivant/existant et pertinent
- `mort` : personnage décédé
- `détruit` : lieu ou objet détruit
- `terminée` / `en_cours` / `échouée` : pour les quêtes
- `inconnu` : disparu ou statut incertain

### QUALITÉ RÉDACTIONNELLE
- Utilise un vocabulaire riche et varié (évite les répétitions)
- Rédige des phrases complètes et fluides
- Sois précis : noms propres, détails concrets, circonstances
- Adopte un ton de chroniqueur médiéval-fantastique

### RÉSOLUTION D'ENTITÉS
Tu disposes de la liste des "Entités déjà connues".
Si une entité du résumé correspond EXACTEMENT (même nom) à une entité connue, utilise ce nom.
Sinon, c'est une nouvelle entité.

---

## EXEMPLE COMPLET

**Résumé session 3 :**
"Le groupe quitte l\'auberge du Sanglier Rouge à l\'aube. Sur la route de Valombre, ils sont attaqués par une bande de gobelins menée par un hobgobelin borgne. Thorn est grièvement blessé mais Lyra parvient à le soigner. Après le combat, ils trouvent sur le hobgobelin une lettre scellée portant le symbole de la Main Noire. À Valombre, le bourgmestre Aldric les accueille froidement. Il leur apprend que des caravanes disparaissent sur la route du col depuis trois semaines. Il promet 200 pièces d\'or à qui résoudra le problème. Le soir, à la taverne, un mystérieux elfe nommé Sylvain leur propose une alliance contre la Main Noire."

**Entités connues :**
- Thorn (pj, actif)
- Lyra (pj, actif)
- Auberge du Sanglier Rouge (lieu, actif)
- Main Noire (faction, actif)

**Extraction attendue :**
```json
{{
  "events": [
    {{
      "order": 1,
      "event_type": "voyage",
      "description": "À l'aube, le groupe quitte l'Auberge du Sanglier Rouge et prend la route en direction de Valombre, cité marchande nichée au pied des montagnes.",
      "entities_involved": ["Thorn", "Lyra", "Auberge du Sanglier Rouge", "Valombre"]
    }},
    {{
      "order": 2,
      "event_type": "combat",
      "description": "Une embuscade éclate sur la route : une bande de gobelins, commandée par un hobgobelin borgne à l'air féroce, fond sur le groupe. L'affrontement est violent et Thorn reçoit une blessure profonde au flanc avant que les assaillants ne soient mis en déroute.",
      "entities_involved": ["Thorn", "Lyra", "Gobelins", "Hobgobelin borgne"]
    }},
    {{
      "order": 3,
      "event_type": "autre",
      "description": "Lyra invoque ses dons de guérisseuse pour refermer la plaie de Thorn. Ses incantations apaisent la douleur et permettent au guerrier de reprendre la route.",
      "entities_involved": ["Lyra", "Thorn"]
    }},
    {{
      "order": 4,
      "event_type": "découverte",
      "description": "En fouillant la dépouille du hobgobelin, le groupe découvre une lettre soigneusement scellée à la cire noire, frappée du sinistre symbole de la Main Noire. Le contenu reste pour l'heure indéchiffré.",
      "entities_involved": ["Hobgobelin borgne", "Main Noire", "Lettre scellée"]
    }},
    {{
      "order": 5,
      "event_type": "voyage",
      "description": "Le groupe atteint enfin les portes de Valombre, cité ceinte de hauts remparts de pierre grise, alors que le soleil décline à l'horizon.",
      "entities_involved": ["Thorn", "Lyra", "Valombre"]
    }},
    {{
      "order": 6,
      "event_type": "rencontre",
      "description": "Le bourgmestre Aldric, un homme austère au regard méfiant, reçoit les aventuriers dans son bureau lambrissé. Son accueil glacial trahit une profonde lassitude face aux étrangers de passage.",
      "entities_involved": ["Aldric", "Thorn", "Lyra"]
    }},
    {{
      "order": 7,
      "event_type": "révélation",
      "description": "Aldric révèle qu'une menace plane sur la région : depuis trois semaines, des caravanes marchandes disparaissent mystérieusement sur la route du col, sans laisser la moindre trace.",
      "entities_involved": ["Aldric", "Route du col"]
    }},
    {{
      "order": 8,
      "event_type": "quête_début",
      "description": "Le bourgmestre promet une récompense de 200 pièces d'or à quiconque éluciderait le mystère des caravanes disparues et mettrait fin à cette menace.",
      "entities_involved": ["Aldric", "Quête des caravanes"]
    }},
    {{
      "order": 9,
      "event_type": "rencontre",
      "description": "Le soir venu, dans la pénombre enfumée de la taverne locale, un elfe au regard perçant nommé Sylvain aborde le groupe. D'une voix basse, il leur propose une alliance contre la Main Noire, suggérant qu'il en sait long sur cette organisation.",
      "entities_involved": ["Sylvain", "Thorn", "Lyra", "Main Noire"]
    }}
  ],
  "entity_updates": [
    {{
      "canonical_name": "Thorn",
      "type": "pj",
      "status": "actif",
      "new_info": "Grièvement blessé au flanc lors de l'embuscade gobeline sur la route de Valombre, puis soigné par Lyra grâce à sa magie curative."
    }},
    {{
      "canonical_name": "Lyra",
      "type": "pj",
      "status": "actif",
      "new_info": "A démontré ses talents de guérisseuse en refermant magiquement la blessure de Thorn après l'embuscade."
    }},
    {{
      "canonical_name": "Valombre",
      "type": "lieu",
      "status": "actif",
      "new_info": "Cité marchande fortifiée au pied des montagnes, ceinte de hauts remparts de pierre grise. Gouvernée par le bourgmestre Aldric. Frappée depuis trois semaines par la disparition inexpliquée de caravanes sur la route du col."
    }},
    {{
      "canonical_name": "Aldric",
      "type": "pnj",
      "status": "actif",
      "new_info": "Bourgmestre de Valombre, homme austère au regard méfiant, visiblement las des étrangers de passage. Offre 200 pièces d'or pour résoudre le mystère des caravanes disparues."
    }},
    {{
      "canonical_name": "Gobelins",
      "type": "créature",
      "status": "mort",
      "new_info": "Bande de pillards ayant tendu une embuscade sur la route de Valombre, menée par un hobgobelin borgne. Vaincus par le groupe."
    }},
    {{
      "canonical_name": "Hobgobelin borgne",
      "type": "créature",
      "status": "mort",
      "new_info": "Chef de la bande de gobelins, reconnaissable à son œil manquant et à son air féroce. Portait une lettre scellée de la Main Noire, suggérant un lien avec cette organisation."
    }},
    {{
      "canonical_name": "Main Noire",
      "type": "faction",
      "status": "actif",
      "new_info": "Une lettre portant leur sceau de cire noire a été retrouvée sur le hobgobelin borgne, indiquant que cette organisation pourrait commanditer des attaques sur les routes de la région."
    }},
    {{
      "canonical_name": "Lettre scellée",
      "type": "objet",
      "status": "actif",
      "new_info": "Document mystérieux découvert sur la dépouille du hobgobelin, scellé à la cire noire et marqué du symbole de la Main Noire. Son contenu n'a pas encore été déchiffré par le groupe."
    }},
    {{
      "canonical_name": "Sylvain",
      "type": "pnj",
      "status": "actif",
      "new_info": "Elfe mystérieux au regard perçant, rencontré dans une taverne de Valombre. Semble détenir des informations sur la Main Noire et propose une alliance au groupe pour lutter contre cette organisation."
    }},
    {{
      "canonical_name": "Quête des caravanes",
      "type": "quête",
      "status": "en_cours",
      "new_info": "Mission confiée par le bourgmestre Aldric de Valombre : enquêter sur la disparition mystérieuse des caravanes marchandes sur la route du col. Récompense promise : 200 pièces d'or."
    }},
    {{
      "canonical_name": "Route du col",
      "type": "lieu",
      "status": "actif",
      "new_info": "Route commerciale menant hors de Valombre à travers les montagnes, devenue dangereuse depuis que des caravanes y disparaissent sans laisser de traces depuis trois semaines."
    }}
  ]
}}
```

---

## FORMAT DE SORTIE JSON

{{
  "events": [
    {{
      "order": <numéro chronologique>,
      "event_type": "<type en français>",
      "description": "<description riche et détaillée>",
      "entities_involved": ["Nom1", "Nom2"]
    }}
  ],
  "entity_updates": [
    {{
      "canonical_name": "<nom exact>",
      "type": "<type>",
      "status": "<statut>",
      "new_info": "<informations détaillées apprises dans cette session>"
    }}
  ]
}}

Extrais maintenant les données de la session {session_number}.
'''

## 7. Recherche d'entités (EXACT MATCH UNIQUEMENT)

In [99]:
import json


def get_relevant_entities(resume: str, top_k: int = 20) -> str:
    """
    Retourne la liste des entités connues pour injection dans le prompt.
    """
    count = entities_collection.count()
    if count == 0:
        return "Aucune entité connue pour le moment (première session)."

    results = entities_collection.query(
        query_texts=[resume],
        n_results=min(top_k, count),
    )

    lines = ["Entités déjà connues dans la base :"]
    for eid, meta in zip(results["ids"][0], results["metadatas"][0]):
        lines.append(f"  - {eid} ({meta['type']}, {meta['status']})")

    return "\n".join(lines)


def find_existing_entity(name: str) -> str | None:
    """
    Match EXACT uniquement (insensible à la casse).

    Les noms sont normalisés dans les résumés,
    donc pas besoin de fuzzy/vector matching.
    """
    count = entities_collection.count()
    if count == 0:
        return None

    existing = entities_collection.get()
    name_lower = name.strip().lower()

    for eid in existing["ids"]:
        if eid.strip().lower() == name_lower:
            return eid

    return None

## 8. Extraction depuis une session

In [100]:
def extract_from_session(session_number: int, resume: str) -> dict:
    """Extrait les événements et entités d'un résumé via le LLM."""
    prompt = EXTRACTION_TEMPLATE.format(
        session_number=session_number,
        resume=resume,
        existing_entities_block=get_relevant_entities(resume),
    )

    raw = generate(prompt, system_prompt=SYSTEM_PROMPT)

    # Nettoyage markdown
    cleaned = raw.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.split("\n", 1)[1]
        cleaned = cleaned.rsplit("```", 1)[0]

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError as e:
        print(f"  [repair] JSON invalide ({e}), tentative de correction...")
        repair_prompt = (
            f"Ce JSON est invalide :\n{cleaned}\n\n"
            f"Corrige-le et renvoie UNIQUEMENT le JSON valide."
        )
        repaired = generate(repair_prompt, system_prompt=SYSTEM_PROMPT)
        repaired = repaired.strip()
        if repaired.startswith("```"):
            repaired = repaired.split("\n", 1)[1]
            repaired = repaired.rsplit("```", 1)[0]
        return json.loads(repaired)

## 9. Ingestion des événements (avec numérotation)

In [101]:
def ingest_events(session_number: int, events: list[dict]):
    """
    Ajoute les événements avec numérotation chronologique.

    ID format: s003_e007 (session 3, événement 7)
    Metadata inclut 'order' pour la chronologie intra-session.
    """
    if not events:
        return

    events_collection.add(
        ids=[f"s{session_number:03d}_e{e.get('order', i+1):03d}" for i, e in enumerate(events)],
        documents=[e["description"] for e in events],
        metadatas=[
            {
                "session": session_number,
                "order": e.get("order", i+1),
                "event_type": e.get("event_type", "autre"),
                "entities": ", ".join(e.get("entities_involved", [])),
            }
            for i, e in enumerate(events)
        ],
    )

## 10. Mise à jour des entités (avec historique par session)

In [102]:
def update_entities(session_number: int, updates: list[dict]):
    """
    Met à jour les entités avec historique par session.

    Format de la fiche :
    "NomEntité : [S1] Première info. [S3] Nouvelle info. [S5] Mise à jour."

    Pas de LLM pour la fusion, juste concaténation structurée.
    """
    if not updates:
        return

    for update in updates:
        name = update.get("canonical_name", "").strip()
        if not name:
            continue

        new_info = update.get("new_info", "").strip()
        if not new_info:
            continue

        canonical = find_existing_entity(name)

        if canonical:
            # --- Entité existante : ajout de l'historique ---
            existing = entities_collection.get(ids=[canonical])
            old_card = existing["documents"][0]
            old_meta = existing["metadatas"][0]

            # Ajouter la nouvelle info avec préfixe de session
            session_prefix = f"[S{session_number}]"

            # Vérifier si cette session a déjà été ajoutée
            if session_prefix not in old_card:
                new_card = f"{old_card} {session_prefix} {new_info}"
            else:
                new_card = old_card  # Session déjà présente, on ne duplique pas

            entities_collection.update(
                ids=[canonical],
                documents=[new_card],
                metadatas=[{
                    "type": update.get("type", old_meta.get("type", "inconnu")),
                    "status": update.get("status", old_meta.get("status", "inconnu")),
                    "first_session": old_meta.get("first_session", session_number),
                    "last_session": session_number,
                }],
            )
            print(f"  [update] '{canonical}'")

        else:
            # --- Nouvelle entité ---
            session_prefix = f"[S{session_number}]"
            anchored_doc = f"{name} : {session_prefix} {new_info}"

            entities_collection.add(
                ids=[name],
                documents=[anchored_doc],
                metadatas=[{
                    "type": update.get("type", "inconnu"),
                    "status": update.get("status", "inconnu"),
                    "first_session": session_number,
                    "last_session": session_number,
                }],
            )
            print(f"  [new] '{name}'")

## 11. Pipeline d'ingestion complet

In [117]:
def is_session_ingested(session_number: int) -> bool:
    """Vérifie si une session a déjà été ingérée."""
    results = events_collection.get(
        where={"session": session_number},
        limit=1,
    )
    return len(results["ids"]) > 0


def ingest_session(session_number: int, resume: str, force: bool = False) -> dict:
    """
    Pipeline complet : extraction -> events -> entities.

    UN SEUL appel LLM pour l'extraction.
    La fusion des entités est une simple concaténation (pas de LLM).
    """
    import time
    start_total = time.time()

    print(f"\n{'='*50}")
    print(f"SESSION {session_number}")
    print(f"{'='*50}")

    if not force and is_session_ingested(session_number):
        print(f"  Session {session_number} déjà ingérée. Utilisez force=True pour réingérer.")
        return {}

    # Extraction
    print("\nExtraction...")
    start = time.time()
    data = extract_from_session(session_number, resume)
    nb_events = len(data.get('events', []))
    nb_entities = len(data.get('entity_updates', []))
    print(f"  {nb_events} événements, {nb_entities} entités")
    print(f"  {time.time() - start:.1f}s")

    # Événements
    print("\nÉvénements...")
    ingest_events(session_number, data.get("events", []))
    print(f"Ingérés")

    # Entités (pas de LLM, concaténation simple)
    print("\nEntités...")
    update_entities(session_number, data.get("entity_updates", []))

    total_time = time.time() - start_total
    print(f"\nSession {session_number} terminée en {total_time:.1f}s")

    return data

import time
import json

def preview_session(session_number: int, resume: str) -> dict:
    """
    ÉTAPE 1 : EXTRACTION SEULE
    Appelle le LLM pour extraire les données, mais N'ÉCRIT RIEN dans la base.
    Affiche le résultat pour vérification.
    """
    print(f"\n{'='*50}")
    print(f"PRÉVISUALISATION SESSION {session_number}")
    print(f"{'='*50}")

    # Vérification informative (ne bloque pas, car on veut peut-être re-tester)
    if is_session_ingested(session_number):
        print(f"NOTE : La session {session_number} existe déjà en base.")

    print("\nExtraction en cours (LLM)...")
    start = time.time()

    # Appel à ta fonction d'extraction existante
    data = extract_from_session(session_number, resume)

    elapsed = time.time() - start
    nb_events = len(data.get('events', []))
    nb_entities = len(data.get('entity_updates', []))

    print(f"Terminé en {elapsed:.1f}s")
    print(f"-> {nb_events} événements détectés")
    print(f"-> {nb_entities} mises à jour d'entités")

    return data

def save_session(session_number: int, data: dict):
    """
    ÉTAPE 2 : INGESTION
    Prend le dictionnaire (vérifié) et l'écrit dans ChromaDB.
    """
    print(f"\n{'='*50}")
    print(f"SAUVEGARDE SESSION {session_number}")
    print(f"{'='*50}")

    if not data:
        print("Aucune donnée à sauvegarder.")
        return

    # Événements
    print("Écriture des événements...")
    ingest_events(session_number, data.get("events", []))

    # Entités
    print("Mise à jour des entités...")
    update_entities(session_number, data.get("entity_updates", []))

    print(f"\nSession {session_number} ingérée avec succès en base.")

## 12. Utilitaires de debug

In [116]:
import pandas as pd
from IPython.display import display, Markdown

def inspect_db():
    """Affiche l'état des deux collections."""
    print("=" * 50)
    print("ENTITÉS")
    print("=" * 50)
    ents = entities_collection.get()
    if not ents["ids"]:
        print("  (vide)")
    for eid, meta, doc in zip(ents["ids"], ents["metadatas"], ents["documents"]):
        print(f"\n  [{eid}]")
        print(f"  Type: {meta['type']} | Statut: {meta['status']} | Sessions: {meta['first_session']}->{meta['last_session']}")
        display_doc = doc if len(doc) < 400 else doc[:400] + "..."
        print(f"  {display_doc}")

    print("\n" + "=" * 50)
    print("ÉVÉNEMENTS")
    print("=" * 50)
    evts = events_collection.get()
    if not evts["ids"]:
        print("  (vide)")

    # Trier par session puis par order
    sorted_evts = sorted(
        zip(evts["ids"], evts["metadatas"], evts["documents"]),
        key=lambda x: (x[1]["session"], x[1].get("order", 0))
    )

    for eid, meta, doc in sorted_evts:
        order = meta.get('order', '?')
        display_doc = doc if len(doc) < 120 else doc[:120] + "..."
        print(f"  [S{meta['session']}.{order}] [{meta['event_type']}] {display_doc}")
        print(f"    Entités: {meta['entities']}")


def reset_db():
    """Supprime et recrée les deux collections. DESTRUCTIF."""
    global events_collection, entities_collection

    chroma_client.delete_collection("events")
    chroma_client.delete_collection("entities")

    events_collection = chroma_client.create_collection(
        name="events",
        embedding_function=embedding_fn,
        metadata={"hnsw:space": "cosine"},
    )
    entities_collection = chroma_client.create_collection(
        name="entities",
        embedding_function=embedding_fn,
        metadata={"hnsw:space": "cosine"},
    )
    print("Base réinitialisée.")


def list_entities():
    """Liste rapide des entités."""
    ents = entities_collection.get()
    print(f"Total: {len(ents['ids'])} entités\n")
    for eid, meta in zip(ents["ids"], ents["metadatas"]):
        print(f"  - {eid} ({meta['type']}, {meta['status']}) [S{meta['first_session']}->{meta['last_session']}]")

def display_preview(data: dict):
    """
    Affiche les données extraites sous forme de tableaux Pandas stylisés.
    """
    if not data:
        print("Aucune donnée à afficher.")
        return

    # --- 1. Affichage des ÉVÉNEMENTS ---
    if "events" in data and data["events"]:
        df_events = pd.DataFrame(data["events"])

        # Nettoyage : convertir la liste d'entités en chaîne de caractères lisible
        if "entities_involved" in df_events.columns:
            df_events["entities_involved"] = df_events["entities_involved"].apply(
                lambda x: ", ".join(x) if isinstance(x, list) else str(x)
            )

        # Réorganisation des colonnes (si elles existent)
        cols = [c for c in ["order", "event_type", "description", "entities_involved"] if c in df_events.columns]
        df_events = df_events[cols]

        display(Markdown("### Séquence des Événements"))
        # Affichage avec alignement à gauche et retour à la ligne pour le texte long
        display(df_events.style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'}))
    else:
        display(Markdown("### Événements : *Aucun*"))

    # --- 2. Affichage des ENTITÉS ---
    if "entity_updates" in data and data["entity_updates"]:
        df_entities = pd.DataFrame(data["entity_updates"])

        # Réorganisation des colonnes
        cols = [c for c in ["canonical_name", "type", "status", "new_info"] if c in df_entities.columns]
        df_entities = df_entities[cols]

        display(Markdown("### Mises à jour des Entités"))
        display(df_entities.style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'}))
    else:
        display(Markdown("### Entités : *Aucune*"))

---

# INGESTION DES SESSIONS

## 13. Session 1 : Des têtes qui valent un sacré paquet

Pour cette première session, on adopte une approche human-in-the-loop :

```
TEXTE BRUT
    |
    v
EXTRACTION PAR LLM
    |
    v
PRE-VISUALISATION (HUMAN-IN-THE-LOOP)
    |
    v
ENREGISTREMENT DANS CHROMADB
```

In [105]:
session_1 = """
Session 1 : Des têtes qui valent un sacré paquet
Date : 29 février
The group of PJs (Heroes) flees following Gorion's death (PNJ). They are joined by Imoen (PNJ), Winthrop's adopted daughter (PNJ), who confirms Gorion's death in the ambush.

On the road to the Northeast, the group meets Elminster (PNJ). The latter presents himself as a member of the Menestrels (Faction) faction and an old friend of Gorion's. He indicates that he is going to Château-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).

The group of PJs (Heroes) flees following Gorion's death (PNJ). They are joined by Imoen (PNJ), Winthrop's adopted daughter (PNJ), who confirms Gorion's death in the ambush.

On the road to the Northeast, the group meets Elminster (PNJ). The latter presents himself as a member of the Menestrels (Faction) faction and an old friend of Gorion's. He indicates that he is going to Château-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).
the ambush.

On the road to the Northeast, the group meets Elminster (PNJ). The latter presents himself as a member of the Menestrels (Faction) faction and an old friend of Gorion's. He indicates that he is going to Château-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).

The group of PJs (Heroes) flees following Gorion's death (PNJ). They are joined by Imoen (PNJ), Winthrop's adopted daughter (PNJ), who confirms Gorion's death in the ambush.

On the road to the Northeast, the group meets Elminster (PNJ). The latter presents himself as a member of the Menestrels (Faction) faction and an old friend of Gorion's. He indicates that he is going to Château-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).

PJ's group intervenes in a convoy attack by goblins. Tungdill (PJ) eliminates the last goblins with an axe. The group rescued a merchant and his son, then escorted the convoy to the Brasamical Inn (Place). À l'Auberge de Brasamical (Lieu), le groupe se sépare. Haldir (PJ) se rend au Temple de la Sagesse pour prier. Le reste du groupe (Ella, Ysabella, Tungdill) se dirige vers la taverne.

l (Place).

The group of PJs (Heroes) flees following Gorion's death (PNJ). They are joined by Imoen (PNJ), Winthrop's adopted daughter (PNJ), who confirms Gorion's death in the ambush.

On the road to the Northeast, the group meets Elminster (PNJ). The latter presents himself as a member of the Menestrels (Faction) faction and an old friend of Gorion's. He indicates that he is going to ChâTeau-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).

PJ's group intervenes in a convoy attack by goblins. Tungdill (PJ) eliminates the last goblins with an axe. The group rescued a merchant and his son, then escorted the convoy to the Brasamical Inn (Place). At the Auberge de Brasamical (Lieu), the group split. Haldir (PJ) goes to the Temple of Wisdom to pray. The rest of the group (Ella, Ysabella, Tungdill) heads to the tavern.

Teau-Suif (Place) to bury Gorion and advises the PCs to take refuge at the Auberge de Brasamical (Place).

PJ's group intervenes in a convoy attack by goblins. Tungdill (PJ) eliminates the last goblins with an axe. The group rescued a merchant and his son, then escorted the convoy to the Brasamical Inn (Place). At the Auberge de Brasamical (Lieu), the group split. Haldir (PJ) goes to the Temple of Wisdom to pray. The rest of the group (Ella, Ysabella, Tungdill) heads to the tavern.

In front of the tavern entrance, a murderer named Marane (PNJ) ambushes the group while searching for the "children of Château-Suif." The fight begins. Marane (PNJ) is neutralized by the PCs and arrested by the guards. Avant l'arrestation, Ella (PJ) subtilise un Contrat d'assassinat (Objet) sur Marane (PNJ). Le document promet 1000 pièces d'or par tête, sans signature de commanditaire.

À l'intérieur de la taverne, Haldir (PJ) rejoint le groupe. La tenancière Gellena (PNJ) présente le groupe à Jaheira (PNJ) et Khalid (PNJ).

Jaheira (PNJ) révèle qu'elle et Khalid (PNJ) sont membres de la faction Les Ménestrels (Faction) et amis de Gorion. Elle suspecte qu'une des guildes de La Porte de Baldur (Lieu) a mis un contrat sur les têtes des PJ. Elle informe que la ville est fermée pour cause de préparatifs de guerre.

Durant la nuit, Ella (PJ) et Ysabella (PJ) surprennent une conversation : des voleurs prévoient de cambrioler la boutique de Kagain (PNJ) à Bérégost (Lieu) pour voler un joyau dans les 3 jours.

Tous les PJ font un rêve où Gorion (PNJ) leur demande de continuer leur route.

Le lendemain matin, le groupe, accompagné de Jaheira (PNJ) et Khalid (PNJ), interroge Marane (PNJ). Ysabella (PJ) utilise sa musique pour faciliter l'aveu. Marane (PNJ) révèle qu'elle a été engagée par la faction Guilde des Voleurs (Faction) de La Porte de Baldur (Lieu). Elle donne l'emplacement du QG (un magasin d'objets magiques) et le mot de passe : \"Fazad\".

Le groupe de PJ décide de partir pour Bérégost (Lieu) afin de prévenir Kagain (PNJ) du cambriolage imminent. Jaheira (PNJ) et Khalid (PNJ) intègrent le groupe pour le voyage.
"""

In [118]:
# Extraction structurée
data_session_1 = preview_session(1, session_1)


PRÉVISUALISATION SESSION 1
NOTE : La session 1 existe déjà en base.

Extraction en cours (LLM)...
Terminé en 22.1s
-> 12 événements détectés
-> 17 mises à jour d'entités


In [119]:
# Affichage pour vérification manuelle
print(display_preview(data_session_1))

### Séquence des Événements

,order,event_type,description,entities_involved
0,1,rencontre,"Les PJ, encore sous le choc de l'embuscade fatale à Gorion, sont rejoints par Imoen, la fille adoptive de Winthrop. Elle confirme la mort tragique de leur protecteur et ami.","Imoen, Gorion, Winthrop, Ella, Ysabella, Tungdill, Haldir"
1,2,rencontre,"Sur la route du Nord-Est, le groupe croise la route d'Elminster, un mage vénérable. Se présentant comme un Ménestrel et un vieil ami de Gorion, il annonce qu'il se rend à Château-Suif pour inhumer le défunt et conseille aux PJ de chercher refuge à l'Auberge de Brasamical.","Elminster, Menestrels, Gorion, Château-Suif, Auberge de Brasamical"
2,3,combat,"Le groupe intervient héroïquement pour défendre un convoi marchand attaqué par des gobelins. Tungdill se distingue en abattant les derniers assaillants avec sa hache, sauvant ainsi le Marchand et son Fils. Le groupe escorte ensuite le convoi jusqu'à l'Auberge de Brasamical.","Tungdill, Goblins, Marchand, Fils du Marchand, Auberge de Brasamical"
3,4,repos,"Arrivé à l'Auberge de Brasamical, le groupe se sépare temporairement. Haldir se dirige vers le Temple de la Sagesse pour un moment de recueillement, tandis qu'Ella, Ysabella et Tungdill se rendent à la taverne de l'auberge.","Auberge de Brasamical, Haldir, Temple de la Sagesse, Ella, Ysabella, Tungdill"
4,5,combat,"Devant l'entrée de la taverne, une meurtrière nommée Marane tend une embuscade au groupe, cherchant explicitement les « enfants de Château-Suif ». L'affrontement est bref, Marane est neutralisée par les PJ et immédiatement arrêtée par les gardes de l'auberge.","Marane, Château-Suif, Ella, Ysabella, Tungdill, Gardes"
5,6,acquisition,"Juste avant que Marane ne soit emmenée par les gardes, Ella fait preuve de discrétion et subtilise un Contrat d'assassinat sur la meurtrière. Le document, non signé, promet une somme colossale de 1000 pièces d'or pour chaque tête des PJ.","Ella, Marane, Contrat d'assassinat"
6,7,rencontre,"À l'intérieur de la taverne, Haldir retrouve le reste du groupe. La tenancière, Gellena, introduit les aventuriers à deux individus : Jaheira et Khalid.","Haldir, Gellena, Jaheira, Khalid"
7,8,révélation,"Jaheira révèle que, tout comme Elminster, elle et Khalid sont des Ménestrels et des amis proches de Gorion. Elle émet l'hypothèse qu'une des Guildes Criminelles de La Porte de Baldur est responsable du contrat d'assassinat. Elle ajoute que la ville est actuellement fermée en raison de préparatifs de guerre.","Jaheira, Khalid, Menestrels, Gorion, Guildes Criminelles, La Porte de Baldur"
8,9,révélation,"Pendant la nuit, Ella et Ysabella surprennent fortuitement une conversation de Voleurs. Elles apprennent qu'un cambriolage est planifié dans les trois jours à Bérégost, visant la boutique de Kagain pour y dérober un Joyau précieux.","Ella, Ysabella, Voleurs, Kagain, Bérégost, Joyau"
9,10,rêve,"Tous les membres du groupe partagent un rêve commun et troublant : l'apparition spectrale de Gorion, qui les exhorte à poursuivre leur route sans se retourner.","Gorion, Ella, Ysabella, Tungdill, Haldir"


### Mises à jour des Entités

,canonical_name,type,status,new_info
0,Haldir,pj,actif,S'est rendu au Temple de la Sagesse à l'Auberge de Brasamical pour prier avant de rejoindre le reste du groupe à la taverne.
1,Tungdill,pj,actif,A fait preuve de bravoure en éliminant les derniers gobelins lors de l'attaque du convoi marchand avec sa hache.
2,Ella,pj,actif,A subtilement dérobé le Contrat d'assassinat sur Marane avant son arrestation. A surpris une conversation de voleurs planifiant un cambriolage à Bérégost.
3,Ysabella,pj,actif,A utilisé sa musique pour faciliter l'aveu de Marane lors de l'interrogatoire. A surpris une conversation de voleurs planifiant un cambriolage à Bérégost.
4,Marane,pnj,actif,Meurtrière engagée par la Guilde des Voleurs de La Porte de Baldur pour assassiner les 'enfants de Château-Suif'. Neutralisée et arrêtée à l'Auberge de Brasamical. A révélé l'emplacement du QG et le mot de passe 'Fazad' sous l'effet de la musique d'Ysabella.
5,Jaheira,pnj,actif,Ménestrel et amie de Gorion. A rejoint le groupe avec Khalid pour le voyage vers Bérégost. Suspecte les Guildes Criminelles de La Porte de Baldur d'être derrière le contrat d'assassinat.
6,Khalid,pnj,actif,Ménestrel et ami de Gorion. A rejoint le groupe avec Jaheira pour le voyage vers Bérégost.
7,Gellena,pnj,actif,Tenancière de l'Auberge de Brasamical. A présenté Jaheira et Khalid au groupe.
8,Kagain,pnj,actif,"Boutiquier ou marchand à Bérégost, cible d'un cambriolage imminent visant un joyau précieux."
9,Guilde des Voleurs,faction,actif,"Organisation criminelle de La Porte de Baldur. Commanditaire du contrat d'assassinat sur les PJ. Leur QG est un magasin d'objets magiques, et leur mot de passe est 'Fazad'."


None


In [110]:
# Ecriture dans la BDD
save_session(1, data_session_1)


SAUVEGARDE SESSION 1
Écriture des événements...
Mise à jour des entités...
  [new] 'Ella'
  [new] 'Ysabella'
  [new] 'Tungdill'
  [new] 'Haldir'
  [new] 'Gorion'
  [new] 'Imoen'
  [new] 'Winthrop'
  [new] 'Elminster'
  [new] 'Marane'
  [new] 'Jaheira'
  [new] 'Khalid'
  [new] 'Gellena'
  [new] 'Marchand'
  [new] 'Fils du Marchand'
  [new] 'Kagain'
  [new] 'Château-Suif'
  [new] 'Auberge de Brasamical'
  [new] 'Temple de la Sagesse'
  [new] 'La Porte de Baldur'
  [new] 'Bérégost'
  [new] 'Menestrels'
  [new] 'Guilde des Voleurs'
  [new] 'Contrat d'assassinat'
  [new] 'Goblins'
  [new] 'Quête de Kagain'
  [new] 'Fazad (Mot de passe)'

Session 1 ingérée avec succès en base.


## 14. Session 2 : Gare aux zombies !

Cependant, il est possible d'entièrement automatiser le processus grace à la bonne fiabilité du pipeline.

In [120]:
session_2 = """
Session 2 : Gare aux zombies !
Date : 28 mars

Avant de quitter l'Auberge de Brasamical (Lieu), un Prêtre de Brasamical (PNJ) informe Haldir (PJ) que la fresque de son dieu s'est brisée, un mauvais présage venant de Diancecht (Dieu). Il conseille de chercher des réponses au Temple de Bérégost (Lieu).

Le groupe de PJ, accompagné de Jaheira (PNJ) et Khalid (PNJ), prend la route vers le sud.
Durant le voyage, Jaheira (PNJ) propose à Ysabella (PJ) et Haldir (PJ) de rejoindre la faction Les Ménestrels (Faction) en tant que guetteurs.
Elle explique également l'histoire du \"Temps des Troubles\" : les Dieux ont été forcés de marcher sur Toril et beaucoup sont morts, affaiblissant grandement Les Ménestrels.

Sur la route, le groupe découvre des avis de recherche : Tungdill (PJ) est mis à prix pour 15 PO par des Gobelins. Un plan vers un camp gobelin se trouve au dos de l'affiche.

Le groupe arrive à Bérégost (Lieu). La ville est occupée par les soldats de la faction Le Poing Enflammé (Faction).
Un Capitaine du Poing Enflammé (PNJ) annonce sur la place publique qu'une guerre se prépare contre le royaume d'Amn (Lieu), accusé de voler le fer et de préparer une attaque contre La Porte de Baldur (Lieu). Les troupes partiront dans 2 jours pour reprendre une forteresse aux Gnolls et marcher sur Nashkel (Lieu).

Le groupe rencontre Othon (PNJ), le maire actuel de Bérégost.
Othon (PNJ) signale des attaques de morts-vivants venant de l'Est et une horde de zombies se dirigeant vers Haute-Haie (Lieu) à l'Ouest. Il offre le logis à l'auberge \"La Gerbe Rouge (Lieu)\".

Concernant la boutique de Kagain (PNJ), ce dernier est absent. Le maire accepte de faire surveiller la boutique.

À l'auberge La Gerbe Rouge (Lieu), le groupe retrouve Firebead (PNJ) (l'homme de Château-Suif). Il cherche un livre de musicologie et suggère de demander au Luthier de Bérégost.

Le groupe se réorganise pour gérer les menaces simultanées :
Jaheira (PNJ) et Khalid (PNJ) quittent le groupe actif pour aider aux défenses militaires de la ville.
Imoen (PNJ) reste en ville pour surveiller la boutique de Kagain (PNJ) contre les cambrioleurs.
Les PJ (Tungdill, Haldir, Ysabella, Ella) partent pour Haute-Haie (Lieu) enquêter sur les zombies.

Arrivés à Haute-Haie (Lieu), Tungdill (PJ) attire involontairement une horde de zombies. Le groupe se réfugie dans le bâtiment principal.

Ils rencontrent Thalantyr (PNJ), un mage nécromancien vivant là.
Thalantyr (PNJ) révèle son passé : manipulé par Cyric (Dieu) et l'ancien maire Roger (PNJ), il a jadis détruit une académie magique rivale pour éliminer la concurrence. Les attaques actuelles sont une vengeance liée à ce passé.
Il donne une quête aux PJ : stopper la menace. Récompense promise : parchemins et identifications gratuites.

Le groupe se rend aux Ruines d'Ulcaster (Lieu), l'ancienne académie détruite.
Ella (PJ) utilise son familier corbeau pour repérer un spectre.
Le groupe discute avec Ulcaster (PNJ) (le fantôme de l'ancien dirigeant). Il explique que le responsable est un nécromancien caché au sous-sol, utilisant une baguette brisée pour relever les morts.

Les PJ descendent dans le sous-sol des ruines pour affronter le nécromancien.
"""

In [121]:
# Ingestion de la Session 2
data_session_2 = ingest_session(2, session_2, force=True)

# Vérification des modifications
display_preview(data_session_2)


SESSION 2

Extraction...
  20 événements, 48 entités
  32.2s

Événements...
Ingérés

Entités...
  [update] 'Haldir'
  [update] 'Auberge de Brasamical'
  [update] 'Jaheira'
  [update] 'Bérégost'
  [update] 'Menestrels'
  [update] 'La Porte de Baldur'
  [update] 'Kagain'
  [update] 'Khalid'
  [update] 'Imoen'
  [update] 'Ella'
  [update] 'Tungdill'
  [update] 'Château-Suif'
  [update] 'Quête de Kagain'
  [new] 'Prêtre de Brasamical'
  [new] 'Diancecht'
  [new] 'Temple de Bérégost'
  [update] 'Ysabella'
  [new] 'Temps des Troubles'
  [new] 'Dieux'
  [new] 'Toril'
  [new] 'Gobelins'
  [new] 'Avis de recherche'
  [new] 'Camp gobelin'
  [new] 'Le Poing Enflammé'
  [new] 'Capitaine du Poing Enflammé'
  [new] 'Amn'
  [new] 'Gnolls'
  [new] 'Nashkel'
  [new] 'Othon'
  [new] 'Morts-vivants'
  [new] 'Zombies'
  [new] 'Haute-Haie'
  [new] 'La Gerbe Rouge'
  [new] 'Firebead'
  [new] 'Livre de musicologie'
  [new] 'Luthier de Bérégost'
  [new] 'Thalantyr'
  [new] 'Cyric'
  [new] 'Roger'
  [new] 'Ru

### Séquence des Événements

,order,event_type,description,entities_involved
0,1,révélation,"Avant de quitter l'Auberge de Brasamical, un Prêtre de Brasamical interpelle Haldir pour lui révéler un funeste présage : la fresque de son dieu s'est brisée, signe d'une colère ou d'une intervention de Diancecht. Le prêtre conseille au groupe de chercher des éclaircissements au Temple de Bérégost.","Auberge de Brasamical, Prêtre de Brasamical, Haldir, Diancecht, Temple de Bérégost"
1,2,voyage,"Le groupe d'aventuriers, accompagné de Jaheira et Khalid, prend la route vers le sud, quittant les environs de Château-Suif pour se diriger vers Bérégost.","Jaheira, Khalid, Château-Suif, Bérégost"
2,3,dialogue,"Durant le voyage, Jaheira propose à Ysabella et Haldir de rejoindre la prestigieuse faction des Ménestrels en tant que guetteurs. Elle leur narre également l'histoire tragique du « Temps des Troubles », période où les Dieux furent contraints de fouler le sol de Toril, entraînant la mort de plusieurs divinités et l'affaiblissement des Ménestrels.","Jaheira, Ysabella, Haldir, Menestrels, Temps des Troubles, Dieux, Toril"
3,4,découverte,Le groupe découvre un avis de recherche affiché sur le bord de la route : Tungdill est mis à prix pour 15 pièces d'or par des Gobelins. Au dos de l'affiche se trouve un plan sommaire indiquant l'emplacement d'un camp gobelin.,"Tungdill, Avis de recherche, Gobelins, Camp gobelin"
4,5,voyage,"Les aventuriers atteignent Bérégost, une ville sous haute tension. Ils constatent que la cité est militairement occupée par les soldats de la faction du Poing Enflammé.","Bérégost, Le Poing Enflammé"
5,6,révélation,"Sur la place publique, un Capitaine du Poing Enflammé harangue la foule, annonçant l'imminence d'une guerre contre le royaume d'Amn. Il accuse Amn de voler le fer et de préparer une attaque contre La Porte de Baldur. Il précise que les troupes partiront sous deux jours pour reprendre une forteresse aux Gnolls et marcher ensuite sur Nashkel.","Capitaine du Poing Enflammé, Le Poing Enflammé, Amn, La Porte de Baldur, Gnolls, Nashkel"
6,7,rencontre,"Le groupe rencontre Othon, l'actuel maire de Bérégost, qui leur offre l'hospitalité à l'auberge locale, La Gerbe Rouge.","Othon, Bérégost, La Gerbe Rouge"
7,8,révélation,"Othon alerte les aventuriers sur de graves menaces locales : des attaques de morts-vivants sont signalées à l'Est, et une horde de zombies se dirige dangereusement vers Haute-Haie, à l'Ouest.","Othon, Morts-vivants, Zombies, Haute-Haie"
8,9,dialogue,"Le maire Othon confirme l'absence de Kagain et accepte de faire surveiller sa boutique par les gardes de la ville, assurant ainsi la continuité de la Quête de Kagain.","Othon, Kagain, Quête de Kagain"
9,10,rencontre,"À l'auberge La Gerbe Rouge, le groupe retrouve Firebead, l'érudit rencontré précédemment à Château-Suif. Il est à la recherche d'un précieux Livre de musicologie et leur suggère de s'adresser au Luthier de Bérégost.","La Gerbe Rouge, Firebead, Château-Suif, Livre de musicologie, Luthier de Bérégost"


### Mises à jour des Entités

,canonical_name,type,status,new_info
0,Haldir,pj,actif,A reçu un avertissement d'un prêtre concernant un mauvais présage de Diancecht. A accepté la proposition de Jaheira de devenir guetteur pour les Ménestrels. Part pour Haute-Haie enquêter sur les zombies.
1,Auberge de Brasamical,lieu,actif,"Lieu de départ de la session 2, où un prêtre a délivré un présage funeste à Haldir."
2,Jaheira,pnj,actif,A recruté Haldir et Ysabella pour les Ménestrels. A quitté le groupe à Bérégost pour aider aux défenses militaires de la ville contre la menace d'Amn.
3,Bérégost,lieu,actif,"Ville occupée par Le Poing Enflammé, menacée par une guerre imminente contre Amn et par des attaques de morts-vivants. Dirigée par le maire Othon."
4,Menestrels,faction,actif,Faction affaiblie par le 'Temps des Troubles'. A recruté Haldir et Ysabella comme guetteurs.
5,La Porte de Baldur,lieu,actif,"Cité menacée par une attaque présumée du royaume d'Amn, selon les déclarations du Poing Enflammé."
6,Kagain,pnj,actif,Absent de sa boutique à Bérégost. Sa boutique est désormais surveillée par Imoen et les gardes du maire Othon.
7,Khalid,pnj,actif,"A quitté le groupe à Bérégost pour aider aux défenses militaires de la ville, aux côtés de Jaheira."
8,Imoen,pnj,actif,Est restée à Bérégost pour surveiller la boutique de Kagain contre les cambrioleurs.
9,Ella,pj,actif,A utilisé son familier corbeau pour repérer un spectre aux Ruines d'Ulcaster. Participe à l'enquête sur les morts-vivants à Haute-Haie.


## 15. Session 3 (template)

In [122]:
session_3 = """
Session 3 : Charme-moi si tu peux
​Date : 25 avril
​Le groupe de PJ (Héros) pénètre dans le sous-sol des Ruines d'Ulcaster (Lieu). Le fantôme d'Ulcaster (PNJ) reste à l'entrée et demande de libérer les âmes de ses anciens collègues.
​Ysabella (PJ) ouvre la marche pour désamorcer les pièges.
Tungdill (PJ) repère un groupe de goules et charge immédiatement. Le combat est difficile mais les PJ éliminent les deux créatures. Le bruit du combat semble avoir alerté d'autres monstres.
​Le groupe explore une salle contenant des cercueils et le sarcophage de Rodrik (PNJ), censé porter les Bracelets de l'Oubli (Objet).
Tungdill (PJ) profane la tombe : le squelette à l'intérieur est brisé et ne porte pas les bracelets.
​Le groupe arrive à une intersection. Ils évitent temporairement la salle de conférence (remplie de squelettes animés) pour se diriger vers une porte verrouillée magiquement et une fontaine.
Énigme de la Fontaine : Une femme squelettique est figée près de la fontaine, derrière laquelle se trouvent trois crânes avec des orbites vides ou garnies de joyaux. En replaçant les joyaux tombés dans l'eau dans le bon ordre chromatique, les PJ résolvent l'énigme. La femme squelettique disparaît (âme libérée) et la porte magique se déverrouille.
​Derrière la porte, les PJ interviennent dans un combat entre des goules et des squelettes. Ils aident les squelettes à vaincre les goules. Une fois le combat fini, les squelettes disparaissent, apaisés.
La pièce contient une statue de Mystra (Dieu) bien conservée et un mur suspect dissimulant une porte secrète nécessitant une clé ou un mécanisme.
​Le groupe emprunte la seule issue restante et rencontre un vieil homme hagard, Basillus (PNJ). Il tient une baguette magique.
Basillus (PNJ), confus, prend les PJ pour des agresseurs, invoque une goule et attaque.
Haldir (PJ) plaque Basillus (PNJ) au sol, brisant sa baguette dans l'action.
Ella (PJ) lance un sort de Charme-Personne, ce qui met fin au combat et calme Basillus (PNJ).
​Interrogatoire et Fouille :
​Basillus (PNJ) est amnésique et porte les Bracelets de l'Oubli (Objet), qui sont identifiés comme maudits.
​Les PJ récupèrent la Baguette de résurrection brisée (Objet) et un Médaillon (Objet) correspondant à la porte secrète.
​Basillus (PNJ) est identifié comme un ancien mage de l'académie.
​Les PJ utilisent le Médaillon (Objet) pour ouvrir le mur secret dans la salle de Mystra.
Ils découvrent une salle au trésor macabre contenant des cadavres d'enfants.
Butin récupéré :
​2000 Pièces d'or (partagées).
​Un Journal (Objet).
​Le Grimoire de Basillus (Objet) (récupéré par Ella).
​Sur le chemin du retour, le groupe entre dans la salle de conférence pour la dernière épreuve.
Énigme de la Salle de Conférence : Des squelettes attendent devant un pupitre. Pour les libérer, il faut lancer un sort d'une école de magie commune.
Ella (PJ) lance un sort d'évocation. L'énigme est résolue, les squelettes disparaissent.
​Ulcaster (PNJ) réapparaît. Il remercie les PJ d'avoir libéré ses confrères, confirme que Basillus (PNJ) était l'un des leurs, puis disparaît définitivement.
​Le groupe décide de retourner voir Thalantyr (PNJ) à Haute-Haie (Lieu) pour tenter de lever la malédiction des bracelets sur Basillus (PNJ).
"""

In [123]:
# Ingestion de la Session 3
data_session_3 = ingest_session(3, session_3, force=True)
inspect_db()


SESSION 3

Extraction...
  22 événements, 28 entités
  30.1s

Événements...
Ingérés

Entités...
  [update] 'Ulcaster'
  [update] 'Quête des morts-vivants'
  [update] 'Ysabella'
  [update] 'Tungdill'
  [update] 'Haldir'
  [update] 'Ella'
  [new] 'Goule'
  [new] 'Rodrik'
  [new] 'Bracelets de l'Oubli'
  [new] 'Salle de conférence'
  [new] 'Squelettes'
  [new] 'Femme squelettique'
  [new] 'Mystra'
  [new] 'Basillus'
  [new] 'Baguette de résurrection brisée'
  [new] 'Médaillon'
  [new] 'Académie'
  [new] 'Salle au trésor macabre'
  [new] 'Journal'
  [new] 'Grimoire de Basillus'
  [update] 'Haute-Haie'
  [new] 'Statue de Mystra'
  [new] 'Mur secret'
  [new] 'Fontaine'
  [new] 'Charme-Personne'
  [new] 'École de magie'
  [new] 'Pièces d'or'
  [new] 'Cadavres d'enfants'

Session 3 terminée en 32.3s
ENTITÉS

  [Ella]
  Type: pj | Statut: actif | Sessions: 1->3
  Ella : [S1] Fuyarde de Château-Suif. A dérobé le Contrat d'assassinat sur Marane. A surpris la conversation concernant le cambriolag

---

# UTILITAIRES

## 16. Visualisation de la base

In [124]:
inspect_db()

ENTITÉS

  [Ella]
  Type: pj | Statut: actif | Sessions: 1->3
  Ella : [S1] Fuyarde de Château-Suif. A dérobé le Contrat d'assassinat sur Marane. A surpris la conversation concernant le cambriolage de Kagain à Bérégost. [S2] A utilisé son familier corbeau pour repérer un spectre aux Ruines d'Ulcaster. Participe à l'enquête sur les morts-vivants à Haute-Haie. [S3] A utilisé un sort de Charme-Personne pour neutraliser Basillus. A résolu l'énigme de la Salle de C...

  [Ysabella]
  Type: pj | Statut: actif | Sessions: 1->3
  Ysabella : [S1] Fuyarde de Château-Suif. A surpris la conversation concernant le cambriolage de Kagain. A utilisé sa musique pour faciliter l'aveu de Marane lors de l'interrogatoire. [S2] A accepté la proposition de Jaheira de devenir guetteuse pour les Ménestrels. Part pour Haute-Haie enquêter sur les zombies. [S3] A ouvert la marche dans les sous-sols des ruines, utilisant son expertise pour dés...

  [Tungdill]
  Type: pj | Statut: actif | Sessions: 1->3
  Tungdill

## 17. Liste des entités

In [125]:
list_entities()

Total: 81 entités

  - Ella (pj, actif) [S1->3]
  - Ysabella (pj, actif) [S1->3]
  - Tungdill (pj, actif) [S1->3]
  - Haldir (pj, actif) [S1->3]
  - Gorion (pnj, mort) [S1->1]
  - Imoen (pnj, actif) [S1->2]
  - Winthrop (pnj, actif) [S1->1]
  - Elminster (pnj, actif) [S1->1]
  - Marane (pnj, actif) [S1->1]
  - Jaheira (pnj, actif) [S1->2]
  - Khalid (pnj, actif) [S1->2]
  - Gellena (pnj, actif) [S1->1]
  - Marchand (pnj, actif) [S1->1]
  - Fils du Marchand (pnj, actif) [S1->1]
  - Kagain (pnj, actif) [S1->2]
  - Château-Suif (lieu, actif) [S1->2]
  - Auberge de Brasamical (lieu, actif) [S1->2]
  - Temple de la Sagesse (lieu, actif) [S1->1]
  - La Porte de Baldur (lieu, actif) [S1->2]
  - Bérégost (lieu, actif) [S1->2]
  - Menestrels (faction, actif) [S1->2]
  - Guilde des Voleurs (faction, actif) [S1->1]
  - Contrat d'assassinat (objet, actif) [S1->1]
  - Goblins (créature, mort) [S1->1]
  - Quête de Kagain (quête, en_cours) [S1->2]
  - Fazad (Mot de passe) (objet, actif) [S1->1]
  - P

In [126]:
# Force l'écriture sur disque. Libère le verrou
del chroma_client
import gc
gc.collect()
print("Client ferme.")

Client ferme.


## 18. Réinitialisation (DESTRUCTIF)

In [108]:
# /!\ ATTENTION : Cette cellule efface toutes les données !
# Décommentez pour exécuter.

#reset_db()
#inspect_db()